In [ ]:
import pandas as pd
import numpy as np


In [ ]:
df=pd.read_csv(r"F:\Adult_Census_Income_Prediction\notebooks\data\adult_income_data.csv",skipinitialspace=True)

In [ ]:
df

In [ ]:
df.isnull().sum()

In [ ]:
# In this dataset missing values have been denoted by '?'
# we are replacing ? with NaN for them to be imputed down the line.
df.replace('?',np.NaN,inplace=True)

In [ ]:
df.isnull().sum()


In [ ]:
df.info()

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
df['Income'].value_counts()
#imbalanced

In [ ]:
df[['workclass','occupation','sex','Income']].loc[(df['workclass'].isna())]

In [ ]:
df.columns

In [ ]:
df['workclass'].value_counts()

In [ ]:
df['education'].value_counts()

In [ ]:
num_cols = df.select_dtypes(exclude='object')

In [ ]:
num_cols.skew()

In [ ]:
imputer=SimpleImputer(strategy='most_frequent')

In [ ]:
df['workclass']=imputer.fit_transform(df[['workclass']]).flatten()
df['occupation']=imputer.fit_transform(df[['occupation']]).flatten()

df['native-country']=imputer.fit_transform(df[['native-country']]).flatten()


In [ ]:
df.isnull().sum()

In [ ]:
df['workclass'].isnull().sum()

In [ ]:
df.shape

In [ ]:
replacement_dict = {**dict.fromkeys([1,2,3,4,5,6,7,8],1),**dict.fromkeys([9,10],2),**dict.fromkeys([11,12],3),
                    **dict.fromkeys([13],4),**dict.fromkeys([14],5),**dict.fromkeys([15,16],6)}


In [ ]:
df['education-num'].replace(replacement_dict,inplace=True)

In [ ]:
df['education-num'].unique()

In [ ]:
df['workclass'].unique()

In [ ]:
workclass_replace= {**dict.fromkeys([ 'Never-worked'],'Without-pay'),**dict.fromkeys(['Self-emp-not-inc'],'Private'),
                    **dict.fromkeys(['State-gov','Local-gov'],'gov')}

In [ ]:
df['workclass'].replace(workclass_replace,inplace=True)

In [ ]:
df['workclass'].unique()

In [ ]:
df['relationship'].unique()

In [ ]:
df['relationship'].replace(['Not-in-family', 'Own-child', 'Unmarried','Other-relative'],'others',inplace=True)

In [ ]:
df['relationship'].unique()

In [ ]:
df['marital-status'].unique()

In [ ]:
df['marital-status'].replace(['Divorced','Married-spouse-absent', 'Separated', 'Married-AF-spouse','Widowed'],'No-spouse',inplace=True)

In [ ]:
df['marital-status'].unique()

In [ ]:
df['race'].unique()

In [ ]:
df['race'].replace('Amer-Indian-Eskimo','Other',inplace=True)

In [ ]:
df['race'].unique()

In [ ]:
df

In [ ]:
df.drop(columns='education',inplace=True)

In [ ]:
data = df.drop(columns='Income')

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder,StandardScaler

In [ ]:
def trial():

    categorical_cols = ['workclass', 'marital-status', 'occupation','relationship', 'race', 'sex', 'native-country']
    numerical_cols = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss','hours-per-week']

    
    #numerical pipeline
    num_pipeline = Pipeline( steps=[  ('imputer',SimpleImputer(strategy='median')),
                                    ('scaler',StandardScaler())
                                   ]
                           )

    #categorical pipeline
    cat_pipeline = Pipeline(
                steps=[

                    ('imputer',SimpleImputer(strategy='most_frequent')),
                    ('onehotcoder',OneHotEncoder(sparse_output=False,drop='first'))
                ]
            )
                            
    preprocessor = ColumnTransformer(
                transformers=        
                [
                    ('num_pipeline',num_pipeline,numerical_cols),
                    ('cat_pipeline',cat_pipeline,categorical_cols)
                ]
            )
    return preprocessor

In [100]:
x=trial()

In [ ]:
x

In [ ]:
data

In [104]:
data_pre =x.fit_transform(data)

In [106]:
cols = x.get_feature_names_out(data.columns)


In [107]:
pipedf=pd.DataFrame(data_pre,columns=cols)

In [ ]:
cat_df =df.select_dtypes(include='object')

In [ ]:
cat_df.drop(columns='Income',inplace=True)

In [ ]:
cat_df

In [ ]:
for col in cat_df.columns:
    cat_df = pd.get_dummies(cat_df, columns=[col], prefix=[col], drop_first=True)

In [ ]:
cat_df

In [108]:
pipedf

,num_pipeline__age,num_pipeline__fnlwgt,num_pipeline__education-num,num_pipeline__capital-gain,num_pipeline__capital-loss,num_pipeline__hours-per-week,cat_pipeline__workclass_Private,cat_pipeline__workclass_Self-emp-inc,cat_pipeline__workclass_Without-pay,cat_pipeline__workclass_gov,...,cat_pipeline__native-country_Portugal,cat_pipeline__native-country_Puerto-Rico,cat_pipeline__native-country_Scotland,cat_pipeline__native-country_South,cat_pipeline__native-country_Taiwan,cat_pipeline__native-country_Thailand,cat_pipeline__native-country_Trinadad&Tobago,cat_pipeline__native-country_United-States,cat_pipeline__native-country_Vietnam,cat_pipeline__native-country_Yugoslavia
0,0.030671,-1.063611,1.173346,0.148453,-0.21666,-0.035429,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.837109,-1.008707,1.173346,-0.145920,-0.21666,-2.222153,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,-0.042642,0.245079,-0.449271,-0.145920,-0.21666,-0.035429,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1.057047,0.425801,-1.260579,-0.145920,-0.21666,-0.035429,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,-0.775768,1.408176,1.173346,-0.145920,-0.21666,-0.035429,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,-0.849080,0.639741,0.362038,-0.145920,-0.21666,-0.197409,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
32557,0.103983,-0.335433,-0.449271,-0.145920,-0.21666,-0.035429,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
32558,1.423610,-0.358777,-0.449271,-0.145920,-0.21666,-0.035429,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
32559,-1.215643,0.110960,-0.449271,-0.145920,-0.21666,-1.655225,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [109]:
s=','.join(pipedf.columns.tolist())

In [111]:
s.replace("num_pipeline__","",).replace("cat_pipeline__","")

'age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass_Private,workclass_Self-emp-inc,workclass_Without-pay,workclass_gov,marital-status_Never-married,marital-status_No-spouse,occupation_Armed-Forces,occupation_Craft-repair,occupation_Exec-managerial,occupation_Farming-fishing,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other-service,occupation_Priv-house-serv,occupation_Prof-specialty,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving,relationship_Wife,relationship_others,race_Black,race_Other,race_White,sex_Male,native-country_Canada,native-country_China,native-country_Columbia,native-country_Cuba,native-country_Dominican-Republic,native-country_Ecuador,native-country_El-Salvador,native-country_England,native-country_France,native-country_Germany,native-country_Greece,native-country_Guatemala,native-country_Haiti,native-country_Holand-Netherlands,native-country_Honduras,native-country_Hong,nat

In [102]:
s

'age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass_Private,workclass_Self-emp-inc,workclass_Without-pay,workclass_gov,marital-status_Never-married,marital-status_No-spouse,occupation_Armed-Forces,occupation_Craft-repair,occupation_Exec-managerial,occupation_Farming-fishing,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other-service,occupation_Priv-house-serv,occupation_Prof-specialty,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving,relationship_Wife,relationship_others,race_Black,race_Other,race_White,sex_Male,native-country_Canada,native-country_China,native-country_Columbia,native-country_Cuba,native-country_Dominican-Republic,native-country_Ecuador,native-country_El-Salvador,native-country_England,native-country_France,native-country_Germany,native-country_Greece,native-country_Guatemala,native-country_Haiti,native-country_Holand-Netherlands,native-country_Honduras,native-country_Hong,nat

In [ ]:
s=s.replace("cat_pipeline__","")

In [ ]:
pipedf.columns=s.split(",")

In [ ]:
pipedf